<p><font size="5"><b>CRIMES ET DELITS EN FRANCE - SIMPLON DEV. DATA </b></font></p>


<p><font size="4"><b> 2-3. PRÉPARATION DONNÉES POUR IMPORT DANS SQL - DONNEES GENDARMERIE </b></font></p>

<p><font size="3"> *SOURCE DATASET : https://www.data.gouv.fr/fr/datasets/crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012/* </font></p>

*© 2020, Gérald Bouget  (<mailto:geraldbouget@gmail.com>)

___

In [1]:
import pandas as pd
import openpyxl
from openpyxl.utils import cell
import os, zipfile
from zipfile import ZipFile
import requests
import collections
#imac:
#os.chdir('/Users/admin/SFR Cloud/Documents/Simplon_DataDev_formation/Projet_final/Datasets/crimes_delits_police_gendarmerie_annuel/')

#macbookpro:
os.chdir('/Users/geraldbouget/SFR Cloud/Documents/Simplon_DataDev_formation/Projet_final/Datasets/crimes_delits_police_gendarmerie_annuel/')


## <div class="alert alert-danger"> WARNING : ERREUR DANS DATASET SUIVANT - VOIR PLUS BAS - CORRIGE ET REMPLACE À PARTIR DE ENTREE 267  </div> 

In [271]:
#chargement workbook excel
wb=openpyxl.load_workbook('crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012_ORIGINAL.xlsx')

In [272]:
# CHARGEMENT FICHIER IDENTIQUE QUE PRÉCEDENT (UTILISATION CODE DIFFERENTE)
wbpd=pd.ExcelFile('crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012_ORIGINAL.xlsx')

## <div class="alert alert-info"> 1. Création dictionnaire Gendarmerie et check doublons </div> 

In [72]:
def gendarmerieDic():
    
    gendarmerie = {}
    for year in wb.sheetnames:
        if 'GN' in year:
            gendarmerie[year]={'departement':[], 'service':[]}

    # récupération des nom de chaque departement de chaque annee à titre unitaire
    for year in gendarmerie:
        for departements in wb[year]['C1':str(cell.get_column_letter(wb[year].max_column)+'1')]:
            for dep in departements:
                if dep.value not in gendarmerie[year]['departement'] and dep.value!=None:
                    gendarmerie[year]['departement'].append(dep.value)
    
        for serv in wb[year]['C2':str(cell.get_column_letter(wb[year].max_column)+'2')]:
            for services in serv:
                if services.value not in gendarmerie[year]['service'] and services.value!=None:
                    gendarmerie[year]['service'].append(services.value)

    return gendarmerie

In [73]:
gendarmerie=gendarmerieDic()
# liste des keys principales dans gendarmerie
liste=list(gendarmerie)

In [74]:
liste

['Services GN 2012',
 'Services GN 2013',
 'Services GN 2014',
 'Services GN 2015',
 'Services GN 2016',
 'Services GN 2017',
 'Services GN 2018',
 'Services GN 2019']

## <div class="alert alert-info"> 2. comparaison /check entre années noms variables multi index (dep  - services) </div> 

In [6]:
def depDiff():
    #comparaison departement entre années
    for i in range(len(liste)-1):
        if set(gendarmerie[liste[i]]['departement']) == set(gendarmerie[liste[i+1]]['departement']):
            print(str(liste[i])+ ' ' + 'a les mêmes départements que' + ' ' +str(liste[i+1]))
        else:
            print(str(liste[i]) + ' n\'a pas les mêmes départements que '+str(liste[i+1]))

In [7]:
depDiff()

Services GN 2012 a les mêmes départements que Services GN 2013
Services GN 2013 a les mêmes départements que Services GN 2014
Services GN 2014 a les mêmes départements que Services GN 2015
Services GN 2015 a les mêmes départements que Services GN 2016
Services GN 2016 n'a pas les mêmes départements que Services GN 2017
Services GN 2017 a les mêmes départements que Services GN 2018
Services GN 2018 a les mêmes départements que Services GN 2019


In [8]:
# departement pas present partout
depmanquant=[g for g in set(gendarmerie[liste[4]]['departement']) if g not in set(gendarmerie[liste[5]]['departement'])]
depmanquant

['986']

In [22]:
def servicesDef():
    #comparaison services entre années
    for i in range(len(liste)-1):
        if set(gendarmerie[liste[i]]['service']) == set(gendarmerie[liste[i+1]]['service']):
            print(str(liste[i])+ ' ' + 'est égale à' + ' ' +str(liste[i+1]))
        else:
            print(str(liste[i])+ ' ' + 'ne correspond pas à' + ' ' +str(liste[i+1]))    

In [23]:
servicesDef()

Services GN 2012 ne correspond pas à Services GN 2013
Services GN 2013 ne correspond pas à Services GN 2014
Services GN 2014 est égale à Services GN 2015
Services GN 2015 ne correspond pas à Services GN 2016
Services GN 2016 ne correspond pas à Services GN 2017
Services GN 2017 ne correspond pas à Services GN 2018
Services GN 2018 ne correspond pas à Services GN 2019


In [273]:
def serviceDoublons():
    # verification doublons dans ligne serivces
    df_doublon=pd.DataFrame({
        'nomDoubon':[],
        'numeroColonne':[],
        'annee':[]
    })

    listeservice=[]
    numerocol=[]
    
    for name in gendarmerie:
        for services in wb[name]['C2':str(openpyxl.utils.cell.get_column_letter(wb[name].max_column)+'2')]:
            for service in services:
                listeservice.append(service.value)
                numerocol.append([service.value, service.column, name])        
            nomdoublon=[item for item, count in collections.Counter(listeservice).items() if count > 1]
            listedoublong=[nc for nc in numerocol if nc[0] in nomdoublon]
            #countdoublon=[count[1] for count in  nomdoublon
            if len(listedoublong)!=0:
                df_doublon=pd.concat([df_doublon,pd.DataFrame(listedoublong).rename(columns={0:'nomDoubon', 
                                                                                                 1:'numeroColonne',
                                                                                                 2:'annee'})])
        #print(pd.DataFrame(listedoublong))
        listeservice=[]
        numerocol=[]
                    
    if len(df_doublon) ==0:
        print('Il n\'y a pas de doublons dans les noms de service sur une année')
    else:
        print('voici la liste des doublons')
        return df_doublon


In [274]:
serviceDoublons()

voici la liste des doublons


,nomDoubon,numeroColonne,annee
0,CGD LA FOA,374.0,Services GN 2017
1,CGD NOUMEA,375.0,Services GN 2017
2,CGD POINDIMIE,376.0,Services GN 2017
3,CGD LA FOA,377.0,Services GN 2017
4,CGD NOUMEA,378.0,Services GN 2017
5,CGD POINDIMIE,379.0,Services GN 2017
0,None,377.0,Services GN 2018
1,None,378.0,Services GN 2018
2,None,379.0,Services GN 2018
0,None,375.0,Services GN 2019


In [275]:
# check libelle et codeindex identiques pour tous les onglets
def checkLibelle(fichier_excel):
    liste_probleme=[]
    df_check=pd.DataFrame({'Code index':[],
                          'libelle':[]})
    # on concat df_check avec le 1er onglet annee 2012
    # pour éviter multiindexage on ne retient que les variables service (header 1 pour gendarmerie)
    df=pd.read_excel(fichier_excel,sheet_name=liste[0], header=1).rename(columns={'Libellé index \\ CGD':'libelle'})
    df=df.iloc[:,[0,1]] # df avec uniquement colonne code index et libelle
    df_check=pd.concat([df_check, df])
    
    # on concat df check avec les années suivantes
    # on sépare les 2 process de concat pour que la comparaison se fasse à partir de la 2eme année
    for sheetname in liste[1::]:
         # on garde uniquement variable service pour éviter multiindexage (header=1 pour gendarmerie)
        df=pd.read_excel(fichier_excel,sheet_name=sheetname, header=1).rename(columns={'Libellé index \\ CGD':'libelle'})
        df=df.iloc[:,[0,1]] # df avec uniquement colonne code index et libelle
        df_check=pd.concat([df_check, df])
        if len(df_check[['Code index', 'libelle']].drop_duplicates(keep=False))>0:
            liste_probleme.append([str(sheetname), df_check[['Code index', 'libelle']].drop_duplicates(keep=False)])
        
    return liste_probleme

In [276]:
checkLibelle(wbpd)

[['Services GN 2019',
       Code index                                            libelle
  95         98.0                                  Index non utilisé
  96        101.0                                  Index non utilisé
  97        102.0  Banqueroutes, abus de biens sociaux et autres ...
  98        103.0                                  Index non utilisé
  99        104.0                                  Index non utilisé
  100       105.0  Prix illicittes, publicité fausse et infractio...
  101       106.0                     Achats et ventes sans factures
  102       107.0  Infractions à l'exercice d'une profession règl...
  103         NaN  Infractions au droit de l'urbanisme et de la c...
  104         NaN                                   Fraudes fiscales
  105         NaN            Autres délits économiques et financiers
  106         NaN                                      Autres délits]]

## <div class="alert alert-danger"> WARNING : DATASET ORGINAL MODIFIE : LE BON DATASET EST LE SUIVANT  </div> 

In [277]:
#chargement workbook excel
wb=openpyxl.load_workbook('crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012-2019-CORRIGE.xlsx')

In [278]:
wbpd=pd.ExcelFile('crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012-2019-CORRIGE.xlsx')

## <div class="alert alert-info"> 3. Structuration table 'service' </div> 

In [426]:
def serviceUnique():
    #creation liste de tous les services uniques depuis 2012
    liste_service_unique = []
    for year in liste: #liste=liste des keys dans dict police
        for serv in gendarmerie[year]['service']:
            if serv not in liste_service_unique:
                liste_service_unique.append(serv)

                #df service intermédiaire
                df_service_def = pd.DataFrame(
                    liste_service_unique).reset_index().rename(
                    columns={'index':'idService', 0:'nomService'})
                #pour être raccord avec autincrementation dans SQL qui commence à 1 et non 0 :
                df_service_def['idService'] = df_service_def['idService']+1
    return df_service_def

In [427]:
df_services_unique=serviceUnique()
df_services_unique.tail()


,idService,nomService
390,391,CGD ANCENIS ST GEREON
391,392,CGD CHATEAU GONTIER SUR MAYENNE
392,393,CGD EVRY COURCOURONNES
393,394,CGD LES ARCHIPELS PAPEETE
394,395,CGD LES ILES DU VENT FAAA


In [428]:
len(df_services_unique)

395

In [382]:
def CombinaisonUniqueService():
    
    v=pd.DataFrame({'numDep':[],
                    'nomService':[]})
    for sheetname in liste: #liste= liste des noms des onglets
        if sheetname!='Services GN 2017':
            # je récupère la liste des nom colonnes (dep-service)
            dep_serv=pd.read_excel(wbpd,sheetname, header=[0,1], nrow=0, index=0).columns[2::]
            v=pd.concat([v, pd.DataFrame({'numDep':[i[0] for i in dep_serv],
                                          'nomService':[i[1] for i in dep_serv]})], axis=0)
        if sheetname=='Services GN 2017': #année où il y a problème doublons de colonnes
            dep_serv=pd.read_excel(wbpd,sheetname, header=[0,1], nrow=0, index=0).columns[2:-3]
            v=pd.concat([v, pd.DataFrame({'numDep':[i[0] for i in dep_serv],
                                              'nomService':[i[1] for i in dep_serv]})], axis=0)
    v=v.drop_duplicates(keep='last').reset_index(drop=True)
    return v

In [383]:
df_combinaisons_uniques=CombinaisonUniqueService()
df_combinaisons_uniques.head()

,numDep,nomService
0,07,CGD PRIVAS
1,33,CGD BORDEAUX-BASTIDE
2,95,CGD ARGENTEUIL
3,987,CGD ILES-DU-VENT-PAPEETE
4,24,CGD RIBERAC


In [384]:
#verification que toujours même nombre de services uniques
len(df_combinaisons_uniques)

395

In [419]:
def tableService():
    df_tableService=df_combinaisons_uniques.copy()
    df_tableService['idAdm']=2 #gendarmerie PK=2
    df_tableService['idDirection']='' #pas de directions données pour la gendarmerie (uniquement pour la police)
    
    return df_tableService

In [420]:
df_table_service=tableService()
df_table_service.head()

,numDep,nomService,idAdm,idDirection
0,07,CGD PRIVAS,2,
1,33,CGD BORDEAUX-BASTIDE,2,
2,95,CGD ARGENTEUIL,2,
3,987,CGD ILES-DU-VENT-PAPEETE,2,
4,24,CGD RIBERAC,2,


In [421]:
len(df_table_service)

395

In [411]:
# export csv pour import SQL
service_to_sql = df_table_service.to_csv('fichiersPourImportSQL/export_services_gendarmerie.csv',
                                        columns=['numDep',
                                                'nomService',
                                                'idDirection',
                                                'idAdm'], index=False)


## <div class="alert alert-info"> 4. Structuration table 'pointage' </div> 

In [422]:
def tablePointage():
    # Création template DF final
    df_pointage=pd.DataFrame({
        'codeIndex':[],
        'nomService':[],
        'nbreInfractions':[],
        'annee':[]
    })
    
    # en raison d'une erreur dans le tableau excel de l'année 2017 (voir verif doublons début notebook)
    # je sépare le processing entre 2017 et les autres années
    listeanneeclean=[name for name in gendarmerie]
    listeanneeclean.remove('Services GN 2017')
    #listeanneeclean
    
    # liste pour accueillir df restructurés par année
    df_pointage_liste = []

    # loop pour aller récupérer dans chaque onglet gendarmerie le nbre 
    # d'infractions / service/ code index ss orme de DF
    for name in listeanneeclean:
        df_temp = pd.read_excel(wbpd,
                  sheet_name=name,
                  header=[1], #ne retenir que la ligne 'services'
                  # on ne garde que code index et pas libelle + services en ligne 
                  # et aggregations nbre d'ifnractions:
                  index_col=0).fillna(0.0, axis=1)
        for i in range(1, len(df_temp.columns)):
            df_temp.iloc[:,i]=df_temp.iloc[:,i].apply(lambda x: int(x))
    
        df_temp=df_temp.drop('Libellé index \ CGD', axis=1).stack().reset_index().rename(
            columns={'Code index':'codeIndex', 'level_1':'nomService', 0:'nbreInfractions'})
        df_temp['annee']=str(name[-4:]) #récupération année à partir noms onglets et mise en colonne
        df_pointage_liste.append(df_temp)
    
    #processing pour année 2017
    df_2017 = pd.read_excel(wbpd,
                  sheet_name='Services GN 2017',
                  header=[1], #ne retenir que la ligne 'services'
                  # on ne garde que code index et pas libelle + services en ligne et 
                  # aggregations nbre d'ifnractions:
                  index_col=0,
                usecols=[i for i in range(0, 376)]) #376 dernière colonne avant doublons
    
    df_2017=df_2017.fillna(0.0, axis=1) #remplacé na values par 0.0 1/2
    
    for i in range(1, len(df_2017.columns)):
        df_2017.iloc[:,i]=df_2017.iloc[:,i].apply(lambda x: int(x)) #conversion float value en int 2/2
        
    #utilisation stack pour avoir services en lignes et nbre d'infraction en colonnes aggrégées
    df_2017=df_2017.drop('Libellé index \ CGD', axis=1).stack().reset_index().rename(
            columns={'Code index':'codeIndex', 'level_1':'nomService', 0:'nbreInfractions'})
    df_2017['annee']='2017' #récupération année à partir noms onglets et mise en colonne
    df_pointage_liste.append(df_2017)
        
        
    # concatener tous les df restructurés de chaque année
    df_pointage=pd.concat(df_pointage_liste)
    
   
    
    return df_pointage
    

In [423]:
df_table_pointage=tablePointage()

In [424]:
df_table_pointage[df_table_pointage.duplicated()]

,codeIndex,nomService,nbreInfractions,annee


In [425]:
df_table_pointage.tail()

,codeIndex,nomService,nbreInfractions,annee
40013,107,CGD ILES DU VENT FAAA,202,2017
40014,107,CGD KONE,42,2017
40015,107,CGD LA FOA,49,2017
40016,107,CGD NOUMEA,129,2017
40017,107,CGD POINDIMIE,16,2017


In [34]:
 # merge entre nom services et idService et drop colonne nomService
#df_pointage_def=df_pointage.merge(df_table_service[['nomService', 'idService', 'idDirection']], 
                                   #how='left', 
                                   #left_on='nomService', 
                                   #right_on='nomService'
                                  #)

In [410]:
# export CSV
pointage_to_sql = df_table_pointage.to_csv('fichiersPourImportSQL/export_pointage_gendarmerie.csv',
                                          columns=['codeIndex',
                                                'nomService',
                                               'nbreInfractions',
                                               'annee'], index=False)
                                                                                                       